### Chapter 16. advanced rag 구축 - 2 multiquery +rag Fusion

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("openai api 키가 없습니다. 한번더 확인 부탁드립니다.")

os.environ['OPENAI_API_KEY'] = openai_api_key

In [11]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate

#### INDEXING ####

loader = WebBaseLoader(
    web_paths=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    ),
)
docs = loader.load()


# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()


In [12]:
# RAG-Fusion: 관련 검색 쿼리 생성

# template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
# Generate multiple search queries related to: {question} \n
# Output (4 queries):"""


template = """당신은 주어진 하나의 질문을 기반으로 여러 검색 쿼리를 생성하는 유용한 조수입니다. \n
다음 질문과 관련된 여러 검색 쿼리를 생성하세요: {question} \n
출력 (4개의 쿼리):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [14]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60, top_n=2):
    """ 
    여러 개의 순위가 매겨진 문서 리스트를 받아, RRF(Reciprocal Rank Fusion) 공식을 사용하여
    문서의 최종 순위를 계산하는 함수입니다. k는 RRF 공식에서 사용되는 선택적 파라미터이며,
    top_n은 반환할 우선순위가 높은 문서의 개수입니다.
    """
    
    # 각 고유한 문서에 대한 점수를 저장할 딕셔너리를 초기화합니다.
    fused_scores = {}

    # 순위가 매겨진 문서 리스트를 순회합니다.
    for docs in results:
        # 리스트 내의 각 문서와 그 문서의 순위를 가져옵니다.
        for rank, doc in enumerate(docs):
            # 문서를 문자열 형식으로 직렬화하여 딕셔너리의 키로 사용합니다 (문서가 JSON 형식으로 직렬화될 수 있다고 가정).
            doc_str = dumps(doc)
            # 해당 문서가 아직 딕셔너리에 없으면 초기 점수 0으로 추가합니다.
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # 문서의 현재 점수를 가져옵니다 (이전에 계산된 점수).
            previous_score = fused_scores[doc_str]
            # RRF 공식을 사용하여 문서의 점수를 업데이트합니다: 1 / (순위 + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # 문서들을 계산된 점수에 따라 내림차순으로 정렬하여 최종적으로 재정렬된 결과를 얻습니다.
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # 재정렬된 결과에서 우선순위가 높은 top_n 개의 문서만 반환합니다.
    return reranked_results[:top_n]




In [16]:
# RAG-Fusion 체인을 구성합니다.
# generate_queries: 질문에 대해 여러 검색 쿼리를 생성합니다.
# retriever.map(): 생성된 쿼리로 관련 문서들을 검색합니다.
# reciprocal_rank_fusion: 검색된 문서들을 RRF 알고리즘을 통해 결합하여 최종 순위를 계산합니다.
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

# 체인을 실행하여 질문에 대한 검색된 문서들을 가져옵니다.
question = "향후 집값에 대해서 알려줘"
docs = retrieval_chain_rag_fusion.invoke({"question": question})

# 검색된 고유 문서들의 개수를 출력합니다.
len(docs)

# 검색된 고유 문서들을 출력합니다.
docs

/var/folders/q0/jb0zfm1n3blbqmnvqpy7pkyc0000gq/T/ipykernel_23048/3854366426.py:29: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score)


[(Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='전격 시행된 정부의 ‘가계부채 관리 강화 방안’에 따라 29일 서울 부동산 시장은 충격에 빠진 모습이다. 시장에선 이번 대책의 파급 효과, 후속 대책 여부 등을 지켜보기 위해 당분간 매도·매수자들이 관망하면서 매매거\n\n\n한겨레\n\n3시간전'),
  0.098378283095364),
 (Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='서울 아파트값 급등에 대응해 정부가 대대적인 대출 조이기에 나선 가운데 정책대출 한도도 크게 줄어들 전망이다. 윤석열 정부 당시 정책대출 수혜 대상을 크게 확대하면서 집값 상승의 불씨를 댕겼다는 평가를 받았는데, 다\n\n\n한겨레\n\n4시간전'),
  0.06504494976203068)]

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """다음 맥락을 바탕으로 질문에 답변하세요:

{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke(question)

'현재의 대출 규제와 가계부채 관리 강화 방안으로 인해 서울 부동산 시장이 충격을 받고 있는 상황입니다. 주택담보대출 한도가 6억원으로 제한되면서 갭투자자들이 어려움을 겪고 있으며, 매도·매수자들이 관망세를 보이고 있습니다. 이러한 상황은 단기적으로 집값에 하락 압력을 가할 수 있습니다. 그러나 향후 집값의 변화는 정부의 추가 대책이나 시장의 반응에 따라 달라질 수 있으므로, 지속적인 모니터링이 필요합니다.'